##Importing 라이브러리

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(42)  # 무조건 42로 세팅!!

In [2]:
!pip install xgboost

In [3]:
from xgboost import XGBClassifier

##파일 불러오기

In [42]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")
submission = pd.read_csv("/content/sample_submission.csv")
print(train.shape, test.shape, submission.shape)

(48199, 13) (7695, 12) (7695, 2)


##데이터 전처리 (Train + Test)

###0)불필요 열 제거

In [43]:
train.drop(columns=['click_id', 'session ID'], inplace=True)

In [44]:
test.drop(columns=['click_id', 'session ID'], inplace=True)

###1) main category 결측치 채우기

In [45]:
#main category 결측치 채우기 --> by clothing model의 앞 문자(A:1.0, B:2.0, C:3.0, P:4.0)
mask = (train["page 1 (main category)"].isna()) #& (train["page 2 (clothing model)"].notna()) &(train["country"] == 12)
# page 2 (clothing model)의 첫 번째 문자 가져와서 매핑 (A -> 1.0, B -> 2.0, C -> 3.0, P -> 4.0)
mapping = {'A': 1.0, 'B': 2.0, 'C': 3.0, 'P': 4.0}
# 변환 적용
train.loc[mask, "page 1 (main category)"] = train.loc[mask, "page 2 (clothing model)"].str[0].map(mapping)
# 변환이 적용된 데이터 개수 확인
print(train.loc[mask, ["page 1 (main category)", "page 2 (clothing model)"]].dropna().shape[0])

31


In [46]:
#main category 결측치 채우기 --> by clothing model의 앞 문자(A:1.0, B:2.0, C:3.0, P:4.0)
mask = (test["page 1 (main category)"].isna()) #& (train["page 2 (clothing model)"].notna()) &(train["country"] == 12)
# page 2 (clothing model)의 첫 번째 문자 가져와서 매핑 (A -> 1.0, B -> 2.0, C -> 3.0, P -> 4.0)
mapping = {'A': 1.0, 'B': 2.0, 'C': 3.0, 'P': 4.0}
# 변환 적용
test.loc[mask, "page 1 (main category)"] = test.loc[mask, "page 2 (clothing model)"].str[0].map(mapping)
# 변환이 적용된 데이터 개수 확인
print(test.loc[mask, ["page 1 (main category)", "page 2 (clothing model)"]].dropna().shape[0])

3


###2)시간과 관련한 데이터 추출

In [47]:
train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['weekday'] = train['date'].dt.weekday
train.drop(columns=['date'], inplace=True)

In [48]:
test['date'] = pd.to_datetime(test['date'])
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['weekday'] = test['date'].dt.weekday
test.drop(columns=['date'], inplace=True)

###3)colour

In [49]:
# 색상 그룹 매핑 (높은 Click 비율 → 낮은 숫자)
color_map = {
    2: 0, 3: 0, 8: 0,  # 가장 높은 Click 비율 (가장 낮은 숫자)
    4: 1, 7: 1, 14: 1,  # 두 번째 그룹
    9: 2, 10: 2, 11: 2, 13: 2,  # 세 번째 그룹
    1: 3, 5: 3, 6: 3, 12: 3  # 가장 낮은 Click 비율 (가장 높은 숫자)
}

# colour 컬럼 변환
train["colour"] = train["colour"].map(color_map)

In [50]:
test["colour"] = test["colour"].map(color_map)

###country target encoding

In [51]:
train["country_encoded"] = train.groupby("country")["Clicked"].transform("mean")

In [52]:
test["country_encoded"] = train.groupby("country")["Clicked"].transform("mean")

###5)price log변환

In [53]:
train['log_price'] = np.log1p(train['price'])
train.drop(columns=['price'], inplace=True)

In [54]:
test['log_price'] = np.log1p(test['price'])
test.drop(columns=['price'], inplace=True)

###6)상품별 인기 지표

In [55]:
product_ctr = train.groupby('page 2 (clothing model)')['Clicked'].mean().reset_index()
product_ctr.columns = ['page 2 (clothing model)', 'product_ctr']

train = train.merge(product_ctr, on='page 2 (clothing model)', how='left')

#page 2 (clothing model) 삭제
train.drop(columns=['page 2 (clothing model)'], inplace=True)

In [56]:
test = test.merge(product_ctr, on='page 2 (clothing model)', how='left')
# Test 데이터에서 product_ctr가 없는 경우, Train의 평균 CTR로 대체
test['product_ctr'].fillna(train['product_ctr'].mean(), inplace=True)

#page 2 (clothing model) 삭제
test.drop(columns=['page 2 (clothing model)'], inplace=True)

In [57]:
train

order  country  page 1 (main category)  colour  location  \
0          1       29                     1.0       3         5   
1          2       29                     1.0       3         6   
2          3       29                     2.0       2         2   
3          4       29                     2.0       3         6   
4          5       29                     2.0       1         3   
...      ...      ...                     ...     ...       ...   
48194     38       29                     2.0       3         5   
48195     39       29                     2.0       0         6   
48196     40       29                     4.0       1         2   
48197     41       29                     4.0       3         1   
48198      1       29                     1.0       0         2   

       model photography  page  Clicked  year  month  day  weekday  \
0                      1     1        1  2024      4    1        0   
1                      1     1        0  2024      4    1        0   
2                      1     1        1  2024      4    1        0   
3                      2     1        0  2024      4    1        0   
4                      2     1        0  2024      4    1        0   
...                  ...   ...      ...   ...    ...  ...      ...   
48194                  2     2        0  2024      4   30        1   
48195                  1     2        0  2024      4   30        1   
48196                  1     1        1  2024      4   30        1   
48197                  2     5        0  2024      4   30        1   
48198                  2     1        0  2024      4   30        1   

       country_encoded  log_price  product_ctr  
0              0.36522   3.688879     0.286611  
1              0.36522   3.713572     0.310112  
2              0.36522   3.850148     0.303116  
3              0.36522   3.433987     0.166227  
4              0.36522   3.737670     0.232394  
...                ...        ...          ...  
48194          0.36522   3.688879     0.067797  
48195          0.36522   3.850148     0.270833  
48196          0.36522   3.912023     0.662539  
48197          0.36522   3.912023     0.203125  
48198          0.36522   3.828641     0.422402  

[48199 rows x 15 columns]

###7)중복되는 데이터 삭제

In [63]:
train.drop(columns=['country'], inplace=True)

In [64]:
test.drop(columns=['country'], inplace=True)

##학습하기
-XGboost 모델 사용하기
-하이퍼파라미터의 영향을 많이 받는 모델이기에 최적의 조합 찾기

In [65]:
X = train.drop('Clicked', axis=1)
y = train['Clicked']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(38559, 13) (9640, 13) (38559,) (9640,)


-

In [66]:
clf=XGBClassifier()
params = {'n_estimators':[50, 100, 200],
          'max_depth':[2,3,4],
          }
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, n_jobs=-1)
grid_cv.fit(X_train, y_train)
print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:
 {'max_depth': 2, 'n_estimators': 50}
최고 예측 정확도: 0.6813


In [ ]:
#전거 = [0.6844-0.6790]

In [71]:
xgb_clf = XGBClassifier(**grid_cv.best_params_)
#[0.6939-0.6647],[]
#xgb_clf = XGBClassifier(n_estimators=100, max_depth=4, learning_rate = 0.05)
xgb_clf.fit(X_train, y_train)

train_pred = xgb_clf.predict(X_train)
val_pred = xgb_clf.predict(X_val)

print("Train ACC : %.4f" % accuracy_score(y_train, train_pred))
print("Val ACC : %.4f" % accuracy_score(y_val, val_pred))

Train ACC : 0.6844
Val ACC : 0.6839


In [68]:
xgb_clf = XGBClassifier(n_estimators=100, max_depth=4, learning_rate = 0.1)
xgb_clf.fit(X_train, y_train)

#0.2[0.6925-0.6805=0.012] #0.1[0.6878-0.6798=0.008] #0.19[0.6932-0.6810=0.0122]
train_pred = xgb_clf.predict(X_train)
val_pred = xgb_clf.predict(X_val)

print("Train ACC : %.4f" % accuracy_score(y_train, train_pred))
print("Val ACC : %.4f" % accuracy_score(y_val, val_pred))

Train ACC : 0.6905
Val ACC : 0.6808


In [70]:
feature_importance = pd.DataFrame(xgb_clf.feature_importances_.reshape((1, -1)), columns=X_train.columns, index=['feature_importance'])
feature_importance

order  page 1 (main category)    colour  location  \
feature_importance  0.194714                    0.04  0.028334  0.006105   

                    model photography      page  year  month       day  \
feature_importance           0.016071  0.023909   0.0    0.0  0.015887   

                     weekday  country_encoded  log_price  product_ctr  
feature_importance  0.015288         0.017197    0.05193     0.590565

##submission.csv 생성

In [72]:
X_test = test

output = xgb_clf.predict(X_test)
assert len(output) == 7695  # sanity check

In [73]:
submission['Clicked'] = output
submission.to_csv('submission.csv', index=False)